In [5]:
from alpaca_trade_api.rest import REST, TimeFrame
import pandas as pd
from dotenv import dotenv_values

In [6]:
BASE_URL = "https://paper-api.alpaca.markets"

config = dotenv_values(".env")
KEY_ID = config['KEY_ID']
SECRET_KEY = config['SECRET_KEY']

In [7]:
# Instantiate REST API Connection
api = REST(key_id=KEY_ID,secret_key=SECRET_KEY,base_url="https://paper-api.alpaca.markets")

# Fetch 1Minute historical bars of Bitcoin
bars = api.get_crypto_bars("BTC/USD", TimeFrame.Minute).df
bars.index = bars.index.tz_convert('America/Los_Angeles')
bars

,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2024-02-26 22:01:00-08:00,56180.7800,56180.7800,56155.9030,1,56158.8000,0.0001,56158.800000,BTC/USD
2024-02-26 22:02:00-08:00,56161.7575,56161.7575,56161.7575,0,56161.7575,0.0000,0.000000,BTC/USD
2024-02-26 22:03:00-08:00,56170.8030,56186.6780,56167.3150,0,56186.6780,0.0000,0.000000,BTC/USD
2024-02-26 22:04:00-08:00,56209.7075,56217.0390,56155.8920,4,56160.0000,0.0004,56184.078416,BTC/USD
2024-02-26 22:05:00-08:00,56210.6450,56210.6450,56210.6450,0,56210.6450,0.0000,0.000000,BTC/USD
...,...,...,...,...,...,...,...,...
2024-02-27 18:20:00-08:00,56860.9810,56860.9810,56834.7500,0,56834.7500,0.0000,0.000000,BTC/USD
2024-02-27 18:21:00-08:00,56873.6320,56873.6320,56873.6320,0,56873.6320,0.0000,0.000000,BTC/USD
2024-02-27 18:24:00-08:00,56904.0845,56904.0845,56883.7450,0,56883.7450,0.0000,0.000000,BTC/USD


In [8]:
from datetime import datetime, timedelta
import math
import time
import string

SYMBOL = 'BTC/USD'
SMA_FAST = 12
SMA_SLOW = 24
QTY_PER_TRADE = 1


def remove_punctuation(text):
    return ''.join(char for char in text if char not in string.punctuation)

# Description is given in the article
def get_pause():
    now = datetime.now()
    next_min = now.replace(second=0, microsecond=0) + timedelta(minutes=1, seconds=1)
    pause = math.ceil((next_min - now).seconds)
    print(f"Sleep for {pause}")
    return pause

# Same as the function in the random version
def get_position(symbol):
    symbol = remove_punctuation(symbol)
    positions = api.list_positions()
    for p in positions:
        if p.symbol == symbol:
            return float(p.qty)
    return 0


# Returns a series with the moving average
def get_sma(series, periods):
    return series.rolling(periods).mean()

# Checks wether we should buy (fast ma > slow ma)
def get_signal(fast, slow):
    print(f"Fast: {fast.iloc[-1]}\nSlow: {slow.iloc[-1]}")
    return fast.iloc[-1] > slow.iloc[-1]

# Get up-to-date 1 minute data from Alpaca and add the moving averages
def get_bars(symbol):
    bars = api.get_crypto_bars(symbol, TimeFrame.Minute).df
    bars[f'sma_fast'] = get_sma(bars.close, SMA_FAST)
    bars[f'sma_slow'] = get_sma(bars.close, SMA_SLOW)
    return bars

while True:
    # GET DATA
    bars = get_bars(symbol=SYMBOL)
    # CHECK POSITIONS
    position = get_position(symbol=SYMBOL)
    should_buy = get_signal(bars.sma_fast,bars.sma_slow)
    print(f"Position: {position} / Should Buy: {should_buy}")
    if position == 0 and should_buy == True:
        # WE BUY ONE BITCOIN
        api.submit_order(SYMBOL, qty=QTY_PER_TRADE, side='buy', time_in_force='ioc')
        print(f'Symbol: {SYMBOL} / Side: BUY / Quantity: {QTY_PER_TRADE}')
    elif position > 0 and should_buy == False:
        # WE SELL ONE BITCOIN
        api.submit_order(SYMBOL, qty=QTY_PER_TRADE, side='sell', time_in_force='ioc')
        print(f'Symbol: {SYMBOL} / Side: SELL / Quantity: {QTY_PER_TRADE}')

    time.sleep(get_pause())
    print("*"*20)

Fast: 56836.92291666666
Slow: 56847.26129166666
Position: 0 / Should Buy: False
Sleep for 47
********************
Fast: 56850.94583333333
Slow: 56851.539625
Position: 0 / Should Buy: False
Sleep for 60
